In [4]:
import tkinter as tk
from tkinter import filedialog, messagebox
import numpy as np
import scipy.io as sio  # 用于保存.mat文件

"""
  * @author: dengyufeng
  * @Created on 2024/8/14 17:20
 """
def load_data(filepath):
    try:
        data = np.load(filepath)  # 假设数据是以.npy格式存储的
        return data
    except Exception as e:
        messagebox.showerror("错误", f"加载数据失败: {e}")
        return None

def filter_trials(epochs_data, threshold, min_trials):
    ntrail = []
    for trail in range(len(epochs_data)):
        if np.max(np.abs(epochs_data[trail])) < threshold:
            ntrail.append(trail)
    
    filtered_data = epochs_data[ntrail, :, :]
    
    # 检查过滤后的试验数量是否不足
    if filtered_data.shape[0] < min_trials:
        print("有效的试验数量不足。")
        return None
    
    return filtered_data

def save_data(data, save_path, file_format):
    try:
        if file_format == 'npy':
            np.save(save_path, data)
        elif file_format == 'mat':
            sio.savemat(save_path, {'filtered_data': data})
        messagebox.showinfo("成功", f"过滤后的数据已保存至: {save_path}")
    except Exception as e:
        messagebox.showerror("错误", f"保存数据失败: {e}")

def filter_data():
    global loaded_data, filtered_data, threshold, min_trials, selected_format, save_directory
    if loaded_data is None:
        messagebox.showwarning("警告", "请先加载数据")
        return

    try:
        threshold_value = float(threshold.get())
        min_trials_value = int(min_trials.get())
    except ValueError:
        messagebox.showwarning("警告", "请输入有效的阈值和试验数量")
        return

    filtered_data = filter_trials(loaded_data, threshold_value, min_trials_value)
    if filtered_data is None:
        messagebox.showwarning("警告", "有效的试验数量不足，未进行保存。")
        return
    
    if save_directory is None:
        messagebox.showwarning("警告", "请选择保存目录")
        return

    save_path = f"{save_directory}/filtered_data_yj.{selected_format.get()}"
    save_data(filtered_data, save_path, selected_format.get())

def batch_process():
    global filtered_data, threshold, min_trials, selected_format, save_directory
    filepaths = filedialog.askopenfilenames(title="选择数据文件", filetypes=[("Numpy Files", "*.npy")])
    if filepaths:
        try:
            threshold_value = float(threshold.get())
            min_trials_value = int(min_trials.get())
        except ValueError:
            messagebox.showwarning("警告", "请输入有效的阈值和试验数量")
            return

        if save_directory is None:
            messagebox.showwarning("警告", "请选择保存目录")
            return

        for filepath in filepaths:
            data = load_data(filepath)
            if data is not None:
                filtered_data = filter_trials(data, threshold_value, min_trials_value)
                if filtered_data is None:
                    messagebox.showwarning("警告", f"文件 {filepath} 有效的试验数量不足，未进行保存。")
                    continue
                
                # 修改保存文件名规则
                base_name = filepath.rsplit('/', 1)[-1].rsplit('.', 1)[0]
                save_path = f"{save_directory}/{base_name}_yz.{selected_format.get()}"
                save_data(filtered_data, save_path, selected_format.get())

def show_data_structure():
    if loaded_data is None:
        messagebox.showwarning("警告", "请先加载数据")
        return

    # 显示处理前数据的结构
    msg = f"处理前数据结构:\n{loaded_data.shape}\n\n"
    
    if 'filtered_data' in globals():
        # 显示处理后数据的结构
        msg += f"处理后数据结构:\n{filtered_data.shape}\n"
    else:
        msg += "尚未处理数据\n"

    messagebox.showinfo("数据结构", msg)

def select_save_directory():
    global save_directory
    save_directory = filedialog.askdirectory(title="选择保存目录")
    if save_directory:
        messagebox.showinfo("成功", f"选择的保存目录为: {save_directory}")
    else:
        save_directory = None

# 创建主窗口
root = tk.Tk()
root.title("数据过滤器")

# 阈值标签
label_threshold = tk.Label(root, text="自定义阈值:")
label_threshold.grid(row=0, column=0, padx=10, pady=10)

# 自定义阈值输入框
threshold = tk.Entry(root, width=10)
threshold.grid(row=0, column=1, padx=10, pady=10)

# 最小试验数量标签
label_min_trials = tk.Label(root, text="最小试验数量:")
label_min_trials.grid(row=0, column=2, padx=10, pady=10)

# 最小试验数量输入框
min_trials = tk.Entry(root, width=10)
min_trials.grid(row=0, column=3, padx=10, pady=10)

# 选择文件按钮
btn_load_file = tk.Button(root, text="选择文件", command=lambda: globals().update(loaded_data=load_data(filedialog.askopenfilename(title="选择数据文件"))))
btn_load_file.grid(row=0, column=4, padx=10, pady=10)

# 保存格式标签
label_format = tk.Label(root, text="保存格式:")
label_format.grid(row=1, column=0, padx=10, pady=10)

# 保存格式选项
selected_format = tk.StringVar(value="npy")
option_format = tk.OptionMenu(root, selected_format, "npy", "mat")
option_format.grid(row=1, column=1, padx=10, pady=10)

# 选择保存目录按钮
btn_select_directory = tk.Button(root, text="选择保存目录", command=select_save_directory)
btn_select_directory.grid(row=1, column=2, padx=10, pady=10)

# 过滤按钮
btn_filter = tk.Button(root, text="过滤数据", command=filter_data)
btn_filter.grid(row=2, column=0, padx=10, pady=10)

# 批量处理按钮
btn_batch_process = tk.Button(root, text="批量处理数据", command=batch_process)
btn_batch_process.grid(row=2, column=1, padx=10, pady=10)

# 显示数据结构按钮
btn_show_structure = tk.Button(root, text="显示数据结构", command=show_data_structure)
btn_show_structure.grid(row=2, column=2, padx=10, pady=10)

# 关闭按钮
btn_close = tk.Button(root, text="关闭", command=root.destroy)
btn_close.grid(row=3, column=0, columnspan=3, padx=10, pady=10)

# 初始化保存目录
save_directory = None

# 进入主事件循环
root.mainloop()
